# find git repo
[here](https://github.com/despresj/ece_colab)

In [6]:
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-24fe6475-c737-f12f-9053-621f8a53c31b)


In [7]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [8]:
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')
  proj_dir = "/content/drive/MyDrive/ece884_project/"
else:
  proj_dir = "../"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
df = pd.read_csv(f"{proj_dir}data_clean/clean.csv")
column_names = df.columns
df = df.to_numpy()

In [10]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(df)
df = scaler.transform(df)

In [11]:
generator = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(200, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(400, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(5000, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(500, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(400, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(200, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(98, activation="sigmoid")
])

In [12]:
discriminator = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(200, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(400, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(5000, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(500, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(400, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(200, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(1, activation="sigmoid") # not correct output dim
])

In [13]:
gan = keras.models.Sequential([generator, discriminator])

discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
discriminator.trainable = False
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

In [14]:
codings_size = 98
batch_size = 32 # number of data points to return
dataset = tf.data.Dataset.from_tensor_slices(df).shuffle(1000)
dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(1)

In [15]:
from tqdm import tqdm

In [16]:
# Although this runs it still may have bugs
def train_gan(gan, dataset, batch_size, codings_size, n_epochs=50, iterations=1000):
    generator, discriminator = gan.layers
    for epoch in tqdm(range(n_epochs)):
        print(f"Epoch {epoch + 1} of {n_epochs}")
        for _ in range(iterations):
            random_index = np.random.permutation(10000)
            X_batch = df[random_index, :]
            # phase 1 - training the discriminator
            noise = tf.random.normal(shape=(10000,98))
            generated_data = generator(noise)
            X_fake_and_real = tf.concat([generated_data, X_batch], axis=0)

            y1 = tf.constant([[0.]] * noise.shape[0] + [[1.]] * X_batch.shape[0])
            y1 = np.reshape(y1, (len(y1), 1))

            discriminator.trainable = True
            discriminator.train_on_batch(X_fake_and_real, y1)
            # phase 2 - training the generator
            noise = tf.random.normal(shape=[batch_size, codings_size])
            y2 = tf.constant([[1.]] * batch_size)
            discriminator.trainable = False
            gan.train_on_batch(noise, y2)

In [ ]:
train_gan(gan, df, batch_size, codings_size, n_epochs=500, iterations=250)

  0%|          | 0/500 [00:00<?, ?it/s]

Epoch 1 of 500


  0%|          | 1/500 [00:44<6:11:22, 44.65s/it]

Epoch 2 of 500


  0%|          | 2/500 [01:18<5:20:11, 38.58s/it]

...

Epoch 5 of 500


  1%|          | 5/500 [03:01<4:49:22, 35.08s/it]

Epoch 6 of 500


  1%|          | 6/500 [03:35<4:45:51, 34.72s/it]

Epoch 7 of 500


In [ ]:
import os
import re
gans_saved = os.listdir(f"{proj_dir}saved_models/")
model_number = [int(re.sub("gan", "", x)) for x in gans_saved]
last_model = max(model_number)
generator.save(f"{proj_dir}saved_models/gan{last_model+1}")

# Need to plot this

In [ ]:
noise = tf.random.normal(shape=[batch_size, codings_size])
generated_data = generator(noise)
generated_data

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.DataFrame(df, columns=column_names)
generated_array = generated_data[0, :]
generated_df = pd.DataFrame(np.reshape(generated_array, (1, 98)), columns=column_names)

In [ ]:
generated_df.loc[:,"hospnum"]

In [ ]:
fig, ax = plt.subplots(7, 14, figsize=(50,50))
ax = ax.ravel()

for i, column in enumerate(df.columns):
    df[column].hist(ax=ax[i], color="skyblue")
    # generated_df[:,column].hist(ax=ax[i], color="red")
    

In [ ]:
fig, ax = plt.subplots(7, 14, figsize=(50,50))
ax = ax.ravel()

for i, column in enumerate(df.columns):
    generated_df[column].hist(ax=ax[i], color="skyblue")